<a href="https://colab.research.google.com/github/Inefra21/Colour-bias-experiment/blob/main/Train_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Models on filtered Cat vs Dogs dataset

# Importing packages

In [ ]:
import os
import math
import shutil
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Loading

In [ ]:
# Mount the drive to access files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Create directory to image folder
# Change the file path
base_dir = '/content/drive/MyDrive/c&d_50'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')

In [ ]:
train_cats_dir = os.path.join(train_dir, 'cats')  # directory with training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')  # directory with training dog pictures
val_cats_dir = os.path.join(val_dir, 'cats')  # directory with validation cat pictures
val_dogs_dir = os.path.join(val_dir, 'dogs')  # directory with validation dog pictures

### Understanding the data

In [ ]:
# Count the images in each folder
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(val_cats_dir))
num_dogs_val = len(os.listdir(val_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

In [ ]:
print('total training cat images:', num_cats_tr)
print('total training dog images:', num_dogs_tr)

print('total validation cat images:', num_cats_val)
print('total validation dog images:', num_dogs_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

# Setting Model Parameters

In [ ]:
# The image generator will produce batches of 100 images of shape 150 pixels by 150 pixels
BATCH_SIZE = 100
IMG_SHAPE  = 150

# Data Augmentation

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
# This is only for visulising the effect of the data augmentation
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

### Applying data augmetation

In [ ]:
# Create image generator which modifies the images
image_gen_train = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_data_gen = image_gen_train.flow_from_directory(batch_size=BATCH_SIZE,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_SHAPE,IMG_SHAPE),
                                                     class_mode='sparse')

In [ ]:
# Visualising the data augmentation
augmented_images = [train_data_gen[0][0][10] for i in range(5)]
plotImages(augmented_images)

### Creating Validation Data generator

In [ ]:
# The data generator for the validation images does not need to use augmentation
image_gen_val = ImageDataGenerator(rescale=1./255)

val_data_gen = image_gen_val.flow_from_directory(batch_size=BATCH_SIZE,
                                                 directory=val_dir,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='sparse')

# Model Creation

In [ ]:
# Load the base model
model = tf.keras.models.load_model(
  '/content/drive/MyDrive/model_base.keras',
  custom_objects={'KerasLayer': hub.KerasLayer})

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

### Model Summary

In [ ]:
# Print model summary
model.summary()

### Train the model

In [ ]:
epochs=100

# Save the model history
history = model.fit(
    train_data_gen,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=len(train_data_gen))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(val_data_gen, steps=int(np.ceil(total_val/100)))
print('Accuracy on test dataset:', test_accuracy)

### Creating validation and loss graphs

In [ ]:
# Plot the accuracy and loss graphs
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Saving and downloading trained model

In [ ]:
# Saving the model
# Change the export path name
export_path = "./model50.keras"
model.save(export_path)

In [ ]:
# Download model
try:
  from google.colab import files
  files.download(export_path)
except ImportError:
  pass